### Use an API Key frrom VisualCrossing to scrape weather data for each track on race days

API site: https://www.visualcrossing.com/weather-api

In [1]:
## Import relevant libraries
import csv
import codecs
import urllib.request
import urllib.error
import sys
import json
from datetime import datetime
import os

import pandas as pd

In [5]:
df = pd.read_csv("/Users/eliserust/Desktop/Papers/Georgetown/Fall 2022/ANLY590/590-Group-Project/data/nyra_2019_complete.csv")

/var/folders/_2/9pk6yj691kx8thgmqggskkk40000gn/T/ipykernel_4101/3706930994.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/eliserust/Desktop/Papers/Georgetown/Fall 2022/ANLY590/590-Group-Project/data/nyra_2019_complete.csv")


In [6]:
## Read in ny complete race data
df = pd.read_csv('data/nyra_2019_complete.csv')

# Set columns
df.columns = ["track_id" ,"race_date" ,"race_number","program_number","trakus_index" ,"latitude" ,"longitude" ,"distance_id" ,"course_type" ,"track_condition" ,"run_up_distance" ,"race_type","post_time" ,"unknown", "weight_carried" ,"jockey" ,"odds" ,"position_at_finish" ]
print(df.columns)

Index(['track_id', 'race_date', 'race_number', 'program_number',
       'trakus_index', 'latitude', 'longitude', 'distance_id', 'course_type',
       'track_condition', 'run_up_distance', 'race_type', 'post_time',
       'unknown', 'weight_carried', 'jockey', 'odds', 'position_at_finish'],
      dtype='object')


In [8]:
## Identify unique locations and geocordinates of each

location = {} # Initialize dictionary of locations
i = 0
for index, row in df.iterrows():
    if i % 10000 == 0:
        print(i)
    if row[0] not in location:
        location[row[0]] = {} # Add location
        location[row[0]][row[1]] = True

    elif row[1] not in location[row[0]]:
        location[row[0]][row[1]] = True
    i +=1
print("Num unqiue tracks: ")
print(len(location))

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
13

In [9]:
# ID unique timestamps per track --> how many races were held at each location?
for key in location:
    print("Num unqiue timestamps per track: ")
    print(len(location[key]))

for key in location:
    lst = []
    print(key)
    for date in location[key].keys():
        lst.append(date)
    print("max date")
    print(max(lst))
    print("min date")
    print(min(lst))

Num unqiue timestamps per track: 
93
Num unqiue timestamps per track: 
85
Num unqiue timestamps per track: 
39
AQU
max date
2019-12-31
min date
2019-01-01
BEL
max date
2019-10-27
min date
2019-04-26
SAR
max date
2019-09-02
min date
2019-07-11


In [11]:
### Identifying latitude/longitude of the three tracks

# ACQ = (40.6729459870339	-73.8275872666695)
# BEL = (40.7158949859935	-73.7192513436757)
# SAR = (43.0720277961227	-73.7701283631861)

coords = {'AQU': ["40.6729459870339",	"-73.8275872666695"],
'BEL': ["40.7158949859935",	"-73.7192513436757"],
'SAR': ["43.0720277961227", "-73.7701283631861"]}

### Scrape data with the API Key

#### Assumptions:
    1. Data set only has 3 truly unique locations (3 race tracks in NY). The geocoordinates of each row are all unique but only differr by a few meters for each location. To save API Calls I averaged the coordinates and used one set for each track.
    2. Weather is aggregated at the 'day' level based on the 'race-date' field in the dataset.

VisualCrossing limits a key to ~ 1000 API calls per day. Used about 950 calls for this.

In [ ]:
### Scrape data
data = []
for key in location: 
    lst = []
    for date in location[key].keys():
        lst.append(date)
    maxdate = max(lst)
    mindate = min(lst)
    weather_api_endpoint="https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/history?"
    try:
        date_time1=datetime.strptime(mindate, '%Y-%m-%d').date()
        date_time2=datetime.strptime(maxdate, '%Y-%m-%d').date()
        latitude = coords[key][0]
        longitude = coords[key][1]
    except ValueError:
        print("Bad date format {}".format(date_time1))
    weather_api_key='GLDSJB5CR78YA4Z7J2AA6E9BW'
    query_params = '&contentType=json&aggregateHours=24&unitGroup=us&includeAstronomy=true&collectStationContributions=true&key={}&startDateTime={}&endDateTime={}&locations={},{}'
    query_params=query_params.format(weather_api_key, date_time1.isoformat(), date_time2.isoformat(),latitude,longitude)

    try:
        response = urllib.request.urlopen(weather_api_endpoint 
                                            +"?"+ query_params)
        data = response.read()
    except Exception:
        print("Error reading from {}"
                .format(weather_api_endpoint +"?"+ query_params))
    weatherData = json.loads(data.decode('utf-8'))
    errorCode=weatherData["errorCode"]
    print(weatherData)
    data.append(weatherData)

    if (errorCode>0):
        print("Error reading from errorCode {}, error={}"
                    .format(weather_api_endpoint +
                        "?"+ query_params, errorCode))
    continue 

In [ ]:
keyList = ['day', 'humidity', 'precipcover', 'precip', 'temp', 'maxt', 'mint', 'cloudcover', 'weathertype', 'snow', 'local']
location_strings = ["40.6729459870339,-73.8275872666695", "40.7158949859935,-73.7192513436757", "43.0720277961227,-73.7701283631861"]
i = 0
keys = ["aqu", "bel", "sar"]
for jsonData in data:
    datadict = {key: [] for key in keyList}
    for item in weatherData['locations'][location_strings[i]]['values']:
        datadict['day'].append(item["datetimeStr"][0:10])
        datadict['humidity'].append(item["humidity"])
        datadict['precipcover'].append(item["precipcover"])
        datadict['precip'].append(item["precip"])
        datadict['temp'].append(item["temp"])
        datadict['maxt'].append(item["maxt"])
        datadict['mint'].append(item["mint"])
        datadict['cloudcover'].append(item["cloudcover"])
        datadict['weathertype'].append(item["weathertype"])
        datadict['snow'].append(item["snow"])
        datadict['local'].append("AQU")
    new = pd.DataFrame.from_dict(datadict)
    new.to_csv(str(keys[i])+".csv", encoding='utf-8')
    i+=1


In [ ]:
aqu = pd.read_csv('aqu.csv')
bel = pd.read_csv('bel.csv')
sar = pd.read_csv('sar.csv')

In [ ]:
dfmerged = pd.concat([aqu, bel, sar], axis=0, ignore_index=True)
dfmerged = dfmerged.drop(dfmerged.columns[[0]],axis = 1)
new_df = pd.merge(df, dfmerged,  how='left', left_on=["track_id","race_date"], right_on = ['local','day'])  
# new_df = pd.merge(new_df, bel,  how='left', left_on=["track_id","race_date"], right_on = ['local','day'])  
# new_df = pd.merge(new_df, sar,  how='left', left_on=["track_id","race_date"], right_on = ['local','day'])  
new_df.to_csv("joined-data.csv", encoding='utf-8')
print("all done!")